In [1]:
import sys
import os
import re

parent_dir = os.path.abspath('../')  
sys.path.append(parent_dir)

from get_benchmark_results_utils import *
from misc import *

In [2]:
full_stats = load_data_structure("../logs/saved_ds/benchmark_stats.yaml")
all_circuit_stats = {}
for type_bm in full_stats:
    for k in full_stats[type_bm]:
        assert k not in all_circuit_stats
        all_circuit_stats[k] = full_stats[type_bm][k]
        all_circuit_stats[k]["type_bench"] = type_bm

In [3]:
ablation_stats_d = dict()
# ablation
for SIMPLIFICATION_flag in ["1", "0"]:
    for BPG_flag in ["1", "0"]:
        for ASSUMPTION_flag in ["1", "0"]:
            flag_str = SIMPLIFICATION_flag + BPG_flag + ASSUMPTION_flag
            cur_max_depth = '4'
            
            log_path = f"../logs/ours/our_logs_{flag_str}_{cur_max_depth}.log"
            log_path_with_contributions = f"./all_logs/ours/our_logs_with_contributions_{flag_str}_{cur_max_depth}.log"
            sr, t = get_overall_average_time_accuracy_from_logs(log_path,all_circuit_stats)
            ablation_stats_d[flag_str] = (sr, t)
                    

In [4]:
ablation_stats_d

{'111': (0.8068181818181819, 6.318026028409091),
 '110': (0.7159090909090908, 3.123936284090909),
 '101': (0.4685714285714285, 10.278984445714286),
 '100': (0.42613636363636365, 2.3767261818181815),
 '011': (0.1477272727272727, 9.455491142045455),
 '010': (0.11931818181818177, 2.403528159090909),
 '001': (0.06285714285714283, 6.78793308),
 '000': (0.07386363636363635, 0.748101340909091)}

In [5]:
ablation_results = ablation_stats_d

def indicator_to_symbol(indicator):
    return '$\checkmark$' if indicator == '1' else '$\\times$'

latex_table = "\\begin{table}[H]\n\\centering\n"
latex_table += "\\begin{tabular}{m{2em}m{2em}m{2em}|cc}\n\\hline\n"
latex_table += "\\circled{4} & \\circled{5} & \\circled{6} & \\textbf{Solved (\\%)} & \\textbf{Time (s)} \\\\ \\hline\n"

for key, (accuracy, time) in ablation_results.items():
    feature_a, feature_b, feature_c = key
    if key == '111':
        row = f"{indicator_to_symbol(feature_a)} & {indicator_to_symbol(feature_b)} & {indicator_to_symbol(feature_c)} & " + "\\textbf{" + f"{accuracy*100:.2f} \%" + "} & \\textbf{" + f"{time:.2f}" + " s} \\\\\n"
    elif key == '000':
        continue
    else:
        row = f"{indicator_to_symbol(feature_a)} & {indicator_to_symbol(feature_b)} & {indicator_to_symbol(feature_c)} & {accuracy*100:.2f} \% & {time:.2f} s \\\\\n"
    
    latex_table += row

latex_table += "\\hline\n"
latex_table += "\\end{tabular}\n"
latex_table += "\\caption{Ablation results. On the left 3 columns, the the removed stages are indicated by a cross (i.e., \\(\\times\\)). A checkmark (i.e., \\(\\checkmark\\))  indicates that a stage is present.}\n"
latex_table += "\\label{tab:ablation}\n"
latex_table += "\\end{table}"

print(latex_table)

\begin{table}[H]
\centering
\begin{tabular}{m{2em}m{2em}m{2em}|cc}
\hline
\circled{4} & \circled{5} & \circled{6} & \textbf{Solved (\%)} & \textbf{Time (s)} \\ \hline
$\checkmark$ & $\checkmark$ & $\checkmark$ & \textbf{80.68 \%} & \textbf{6.32 s} \\
$\checkmark$ & $\checkmark$ & $\times$ & 71.59 \% & 3.12 s \\
$\checkmark$ & $\times$ & $\checkmark$ & 46.86 \% & 10.28 s \\
$\checkmark$ & $\times$ & $\times$ & 42.61 \% & 2.38 s \\
$\times$ & $\checkmark$ & $\checkmark$ & 14.77 \% & 9.46 s \\
$\times$ & $\checkmark$ & $\times$ & 11.93 \% & 2.40 s \\
$\times$ & $\times$ & $\checkmark$ & 6.29 \% & 6.79 s \\
\hline
\end{tabular}
\caption{Ablation results. On the left 3 columns, the the removed stages are indicated by a cross (i.e., \(\times\)). A checkmark (i.e., \(\checkmark\))  indicates that a stage is present.}
\label{tab:ablation}
\end{table}
